In [ ]:
import gymnasium as gym
import numpy as np
import os
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback

class TrainingLogger(BaseCallback):
    def __init__(self, log_filepath="default2.txt", save_dir="model_checkpoints", 
                 save_freq=50000, model_prefix="halfcheetah_ppo2"):
        super().__init__(verbose=0)
        self.log_filepath = log_filepath
        self.save_dir = save_dir
        self.save_freq = save_freq
        self.model_prefix = model_prefix
        self.episode_total_reward = 0
        self.control_costs = 0
        self.last_save = 0
        
        # Create save directory if it doesn't exist
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        
    def _on_step(self):
        # Save model at regular intervals
        if self.num_timesteps >= self.last_save + self.save_freq:
            try:
                # Save the model
                model_path = os.path.join(self.save_dir, 
                                        f"{self.model_prefix}_{self.num_timesteps}")
                self.model.save(model_path)
                print(f"Model saved at {model_path}")
                self.last_save = self.num_timesteps
            except Exception as e:
                print(f"Error saving model at timestep {self.num_timesteps}: {e}")
        
        # Log rewards and metrics
        current_reward = self.locals.get("rewards")[0] if self.locals.get("rewards") is not None else 0
        self.episode_total_reward += current_reward
        
        environment_info = self.locals.get("infos")[0] if self.locals.get("infos") is not None else {}
        episode_done = self.locals.get("dones")[0] if self.locals.get("dones") is not None else False
        
        # Track control costs
        if "reward_ctrl" in environment_info:
            self.control_costs += abs(environment_info["reward_ctrl"])
        
        if episode_done:
            distance_traveled = environment_info.get("x_position", 0)
            with open(self.log_filepath, "a") as log_file:
                log_file.write(f"{self.num_timesteps},{self.episode_total_reward:.4f},{distance_traveled:.4f},{self.control_costs:.4f}\n")
            self.episode_total_reward = 0
            self.control_costs = 0
        return True
    
    def _on_training_start(self):
        with open(self.log_filepath, "w") as log_file:
            log_file.write("timestep,reward,distance,control_cost\n")

def train_halfcheetah_with_ppo():
    try:
        # Setup environment - using standard environment without reward modification
        print("Creating HalfCheetah environment...")
        cheetah_environment = gym.make("HalfCheetah-v4")

        # Initialize PPO model with default hyperparameters
        print("Initializing PPO model...")
        training_model = PPO(
            policy="MlpPolicy",
            env=cheetah_environment,
            verbose=1
        )

        # Setup logger with checkpoint saving capability
        print("Setting up training logger...")
        progress_logger = TrainingLogger(
            log_filepath="halfcheetah_ppo_training2.txt",
            save_dir="model_checkpoints",
            save_freq=50000,
            model_prefix="halfcheetah_ppo2"
        )

        # Train the model
        print("Starting training for 1,000,000 timesteps...")
        training_model.learn(total_timesteps=1000000, callback=progress_logger)

        # Save the final model
        print("Saving final trained model...")
        training_model.save("halfcheetah_ppo_final2")

        print("Training completed successfully!")
        print(f"Training logs saved to {progress_logger.log_filepath}")
        print(f"Model checkpoints saved in {progress_logger.save_dir}")
        
    except Exception as e:
        print(f"Training error occurred: {str(e)}")
        print("Training was interrupted, but intermediate models should be saved.")

if __name__ == "__main__":
    train_halfcheetah_with_ppo()

Creating HalfCheetah environment...


/home/phalle.y/.local/lib/python3.8/site-packages/gymnasium/envs/registration.py:517: DeprecationWarning: WARN: The environment HalfCheetah-v4 is out of date. You should consider upgrading to version `v5`.
  logger.deprecation(


Initializing PPO model...
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/phalle.y/.local/lib/python3.8/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


Setting up training logger...
Starting training for 1,000,000 timesteps...
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -345     |
| time/              |          |
|    fps             | 689      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1e+03       |
|    ep_rew_mean          | -343        |
| time/                   |             |
|    fps                  | 524         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010030727 |
|    clip_fraction        | 0.109       |
|    clip_range           | 0.2         |
|    entropy_loss         | -8.5       